# Week 03 – Unboxing Methods

In this notebook we'll be training a black-box model to the dataset which we will then explain using different post-hoc explanation methods.

Make sure to install the required packages. You can use the lines below to set up your environment. You may also use the environment of last week and install the missing packages in there.
 
```
# navigate to parent directory using cd and activate environment
cd <path-to-week3>

# create and activate new environment
# on Mac
python3.11 -m venv week3_venv
source week3_venv/bin/activate

# on Windows
python3.11 -m venv week3_venv
#or 
py -3.11 -m venv week3_venv
week3_venv\Scripts\activate

# install packages
pip install lime shap torch grad-cam

# install jupyter kernel make environment available as kernel 
pip install ipykernel
python -m ipykernel install --user --name=week3_venv

# close environment
deactivate
```
**import packages**

In [4]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score
import os
import sys
import matplotlib.pyplot as plt
import random

---

# Tabular data – Titanic dataset

## Load the dataset

We are using the Statlog (German Credit Data) dataset (Dheeru Dua and Casey Graff. UCI machine learning repository, 2017. URL [http://archive.ics.uci.edu](http://archive.ics.uci.edu)). The German Credit dataset classifies people described by a set of 20 features as good or bad credit risk.

Make sure to save the dataset in the parent directory or adjust the file path below.

We are directly loading the pre-processed data sets. For some algorithms, we require the data to be in binary form. Hence, we have two versions of X: `X_train` and `X_test`, with continous features and one-hot encoded categorical features, and `X_train_bin` and `X_test_bin`, where all features have been one-hot encoded. For this, continuous features were first transformed into categories. Check the code in `./01_intro/data_prep.ipynb` for more details on pre-processing.

In [ ]:
# X_train 
X_train = pd.read_csv('../datasets/titanic/encoded_titanic_X_train.csv')

# X_test
X_test = pd.read_csv('../datasets/titanic/encoded_titanic_X_test.csv')

# y_train and y_test
y_train = pd.read_csv('../datasets/titanic/titanic_y_train.csv')
y_test = pd.read_csv('../datasets/titanic/titanic_y_test.csv')

# take a look at the data
X_train.head()

## About the dataset

- `Age` – age of a person in years (int)
- `SibSp` – the number of siblings or spouse of a person **onboard** (int)
- `Parch` – the number of parents or children of a person **onbard** (int)
- `Fare` – ticket price (float)
- `Sex` – sex of a person (categorical/binary)
- `Embarked` – location where the traveler mounted from. There are three possible values — Southampton, Cherbourg, and Queenstown (categorical)
- `Pclass` – passenger division into class 1, 2, and 3 (categorical)
- `Survived` – whether person survived the sinking of the ship (binary). Less than 40% survived. This is the **outcome** to predict. 


From the original dataset and from preprocessing the data, we know the following about the **categorical features**:

- `Sex` has two values `['female','male']`, which were encoded `[0,1]`, respectively. Then, after applying one-hot encoding, we have `Sex_1` which indicates `male` if 1, `female` otherwise.
- `Embarked` has three values `['C', 'Q', 'S']`, which were encoded `[0,1,2]`, respectively. Hence,
    - `Embarked_1 = 1` indicates `Q` 
    - `Embarked_2 = 1` indicates `S`
    - `Embarked_1 = 0` and `Embarked_2 = 0` indicated `C`
- `Pclass` has three values `[1,2,3]`, which were encoded `[0,1,2]`, respectively. Hence, after encoding, we have:
    - `Pclass_1 = 1` indicates `2`
    - `Pclass_2 = 1` indicates `3`
    - `Pclass_1 = 0` and `Pclass_2 = 0` indicates `1`

In [6]:
# save feature names
feature_names = list(X_train.columns)

# create a dataframe to save y values in 
y_results = pd.DataFrame()
y_results['y_test'] = y_test

---

# 1. Training a black-box model

We will train a complex, uninterpretable model to the dataset. For the sake of this tutorial, we've chosen to train a Gradient Boosting classifier. We'll use `scikit-learn`'s implementation.

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

We are performig grid search and 5-fold cross validation to determine the best combinations of values for the parameters `n_estimators` and `max_depth`, from the set {100,150,200,250,300} and {2,3,5}, respectively.

In [ ]:
# performing grid search to find the best parameter values
pgrid = {'n_estimators':[100,150,200,250,300],
        'max_depth':[2,3,5]}

gb = GradientBoostingClassifier(random_state=0)
gcv = GridSearchCV(estimator=gb, param_grid=pgrid, n_jobs=1, cv=5, verbose=1, refit=True)
gcv_fit = gcv.fit(X_train, y_train)
gb = gcv_fit.best_estimator_
y_results['gb_pred'] = gb.predict(X_test)
y_results['gb_pred_prob'] = gb.predict_proba(X_test)[:, 1]
print(gb)

## inspect performance

In [ ]:
# Confusion matrix
cm = pd.crosstab(y_results['y_test'], y_results['gb_pred'])
print ("Confusion matrix : \n", cm)

print('\nAccuracy  = %.4f' % accuracy_score(y_results['y_test'], y_results['gb_pred']))
print('F1 score  = %.4f' % f1_score(y_results['y_test'], y_results['gb_pred']))
num_leaves = sum(tree.tree_.n_leaves for tree in gb.estimators_.reshape(-1))           
print('Number of rules = %.0f' % (num_leaves))

### Questions

**Q 1.1 – Which parameters seem to have worked best according to the grid search?**

...

**Q 1.2 – Evaluate the model in terms of performance and interpretability.**

...

---
---
# 2. LIME

Local interpretable model-agnostic explanations (LIME) was proposed by [Ribeiro et al. (2016)](https://dl.acm.org/doi/abs/10.1145/2939672.2939778) and implemented in the package `lime`.

In [13]:
from lime import lime_tabular

In [80]:
# initiate explainer instance
explainer = lime_tabular.LimeTabularExplainer(np.array(X_train), mode='classification',
                                              training_labels=list(y_train['Survived'].unique()),
                                              feature_names = feature_names,random_state=0,
                                              categorical_features=[4,5,6,7,8])

### LOCAL

In [81]:
i = 30
exp_gb = explainer.explain_instance(np.array(X_test.iloc[i,:]), gb.predict_proba, num_features=5)

In [ ]:
print(f'True and predicted values for sample at index {i}:')
print(round(y_results.loc[i,:],2))
print('\n')
exp_gb.show_in_notebook(show_table=True)

### Questions

**Q 2.1 – Take a look at the local explanation for sample with index 30 of the test set. How would you interpret this explanation?.**

The sample got predicted as 0 (not survived), with a probablity of 0.69. We can see that the feature with the highest impact is `Sex_1`, for which the sample has a value of 1, so our sample is male, which reduces their chance of survival quite a lot. On the other hand, this person didn't travel third class (Pclass_2 = 0) and paid a relatively higher fare, which both increases his chance of survival a little bit.

### GLOBAL

In [83]:
from lime import submodular_pick

sp_exp = submodular_pick.SubmodularPick(explainer, 
                                        np.array(X_test),
                                        gb.predict_proba, 
                                        num_features=5,
                                        num_exps_desired=5)

In [ ]:
[exp.show_in_notebook() for exp in sp_exp.sp_explanations]
print('SP-LIME Explanations.')

### Questions

**Q 2.2 – Explain what we're seeing here as global explanation.**

...

---
---

# 3. SHAP

Based on Shapley values from coalitional game theory, SHAP is a model-agnostic method that assigns feature contributions and was introduced by [Lundberg et al. (2017)](https://proceedings.neurips.cc/paper/2017/hash/8a20a8621978632d76c43dfd28b67767-Abstract.html). Besides LIME it belongs to the most well known post-hoc explanation methods and is implemented in the package `shap`.

In [84]:
import shap

explainer = shap.TreeExplainer(gb, X_train)
shap_values = explainer(X_test)

### GLOBAL

In [ ]:
shap.summary_plot(shap_values, X_test)

### Questions

**Q 3.1 – Explain what the summary plot shows and how it can be interpreted.**

...

### LOCAL

There are multiple ways of displaying the feature importances produced by SHAP. Here we show waterfall and force plots.

In [ ]:
print(f'True and predicted values for sample at index {i}:')
print(round(y_results.loc[i,:],2))

In [ ]:
# shap
print('----\nSHAP waterfall plot\n----')
shap.plots.waterfall(shap_values[30,:])

In [ ]:
print('----\nSHAP force plot\n----')
shap.initjs()
shap.force_plot(explainer.expected_value,
                              shap_values[i,:].values,
                              X_test.iloc[i,:])


### Questions

**Q 3.1 – Take a look at the waterfall plot and the force plot. Describe what they show, whether they differ in the information they provide and how you can interpret both.**

...

**Q 3.2 – Is there a way to display only some of the features, let's say the top 5? If yes, how?**

...

---
---

# Image data – Fashion MNIST data

# 1. SHAP on images

## Load the data

In [90]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

In [134]:
X_train = pd.read_csv('../datasets/fashion_mnist/fashion_mnist_X_train.csv', dtype = np.float32)
y_train = pd.read_csv('../datasets/fashion_mnist/fashion_mnist_y_train.csv', dtype = np.float32)
X_test = pd.read_csv('../datasets/fashion_mnist/fashion_mnist_X_test.csv', dtype = np.float32)
y_test = pd.read_csv('../datasets/fashion_mnist/fashion_mnist_y_test.csv',dtype = np.float32)

X_train = X_train.values/255
y_train = y_train.values.flatten()
X_test = X_test.values/255
y_test = y_test.values.flatten()

X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train).type(torch.LongTensor) # data type is long

X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test).type(torch.LongTensor) # data type is long

batch_size = 128
num_epochs = 5

train = torch.utils.data.TensorDataset(X_train, y_train)
test = torch.utils.data.TensorDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)

In [ ]:
labels_dict = {
    0:'T-shirt/top',
    1:'Trouser',
    2:'Pullover',
    3:'Dress',
    4:'Coat',
    5:'Sandal',
    6:'Shirt',
    7:'Sneaker',
    8:'Bag',
    9:'Ankle boot'
}

batch = next(iter(train_loader))
images, labels = batch

# choose random indices for the images
random.seed(0)
random_indices = random.sample(range(len(images)), 5)

# loop through the random indices and save the corresponding images & labels
imgs, lbls = [], []
for i, idx in enumerate(random_indices):
    # Access the image from the dataset along with its label
    imgs.append(images[idx])
    lbls.append(labels[idx])

# plot example images
fig, axes = plt.subplots(1,5, figsize=(8,5))   
for i, img in enumerate(imgs):
    test = img.view(-1, 1, 28, 28)
    axes[i].imshow(-test.squeeze(), cmap='gray')
    axes[i].set_title(f'{labels_dict[lbls[i].item()]}')
    axes[i].axis('off')

## train a convolutional neural network (CNN)

In [136]:
from torch.autograd import Variable

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(32 * 4 * 4, 10) 
    
    def forward(self, x):
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)
        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out

In [137]:
# random seed
random_seed = 1
torch.manual_seed(random_seed)

# initialize model 
model = CNNModel()

# specifiy error as cross entropy loss 
error = nn.CrossEntropyLoss()

# SGD Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# CNN model training
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

for epoch in range(num_epochs):
#     print('Epoch: {}'.format(epoch))
    model.train()
    for i, (images, labels) in enumerate(train_loader):

        train = Variable(images.view(-1, 1, 28, 28))
        labels = Variable(labels)
        
        
        optimizer.zero_grad() # clear gradients
        outputs = model(train) # forward propagation
        loss = error(outputs, labels) # calculate softmax and cross entropy loss
        loss.backward() # calculating gradients
        optimizer.step() # update parameters
        
        count += 1
        
        if i % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch + 1, i * len(images), len(train_loader.dataset),
                100. * i / len(train_loader), loss.item()))

In [118]:
# # save model
# PATH = '../03_unboxing/models/fashion_mnist_net_trained.pth'
# torch.save(model.state_dict(), PATH)

## test performance on test set

In [ ]:
# # load the trained model if necessary
# PATH = '../03_unboxing/models/fashion_mnist_net_trained.pth'
# model = CNNModel()
# model.load_state_dict(torch.load(PATH))

In [ ]:
# Predict test dataset
model.eval()
correct = 0
test_loss = 0
with torch.no_grad():
    for images, labels in test_loader:
        test = Variable(images.view(-1, 1, 28, 28))
        outputs = model(test) # forward propagation
        test_loss += error(outputs, labels.flatten()).item()  # sum up batch loss
        predicted = torch.max(outputs.data, 1)[1] # get predicted label from the maximum value
        probs = F.softmax(outputs) # get all predicted probabilities
        pred_probs = probs[0][predicted] # get probability of predicted label
        correct += (predicted == labels).sum() # total correct predictions

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

### inspect prediction accuracy per class

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for images, labels in test_loader:
        # images, labels = data
        test = Variable(images.view(-1, 1, 28, 28))
        outputs = model(test)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[label.item()] += 1
            total_pred[label.item()] += 1

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classes[classname]:5s} is {accuracy:.1f} %')

## inspect SHAP values

In [ ]:
import shap 

batch = next(iter(test_loader))
images, labels = batch
images = images.view(-1, 1, 28, 28)

# choose random indices for the images
random.seed(0)
random_indices = random.sample(range(len(images)), 10)

# loop through the random indices and save the corresponding images & labels
imgs, lbls = [], []
for i, idx in enumerate(random_indices):
    # get image from the dataset along with its label
    imgs.append(images[idx])
    lbls.append(labels[idx])

background = images[:100]
test_images= images[100:110]
test_labels= labels[100:110]

e = shap.DeepExplainer(model, background)

for i in range(len(imgs)):
    test = imgs[i].view(-1, 1, 28, 28)
    shap_values = e.shap_values(test)

    test_numpy = np.transpose(test.numpy(), (0, 2, 3, 1))

    out = model(test) # forward propagation
    predicted = torch.max(out.data, 1)[1] # get predicted label from the maximum value
    probs = F.softmax(out).detach() # get probabilities for each class
    pred_probs = probs[0][predicted] # get predicted probability for predicted label

    print('--------\n')
    print("Actual Class: {}, Predicted Class: {}, Predicted Probability: {:.4f}\n".format(
        labels_dict[lbls[i].item()],
        labels_dict[predicted.item()],
        pred_probs.item()
    ))

    for cls in labels_dict.keys():
        print('Prob for class {}: {:.4f}'.format(
            labels_dict[cls],
            probs[0][cls]
        ))

    shap_numpy = [np.transpose(shap_values[0, ..., i], (1, 2, 0)).reshape(1, 28, 28, 1) for i in range(shap_values.shape[-1])]
    shap.image_plot(shap_numpy, -test_numpy, labels=list(labels_dict.values()))


**If you have older shap versions (e.g. 0.44.1), try the below code:**

In [ ]:
# import shap 

# batch = next(iter(test_loader))
# images, labels = batch
# images = images.view(-1, 1, 28, 28)

# # choose random indices for the images
# random.seed(0)
# random_indices = random.sample(range(len(images)), 10)

# # loop through the random indices and save the corresponding images & labels
# imgs, lbls = [], []
# for i, idx in enumerate(random_indices):
#     # get image from the dataset along with its label
#     imgs.append(images[idx])
#     lbls.append(labels[idx])

# background = images[:100]
# test_images= images[100:110]
# test_labels= labels[100:110]

# e = shap.DeepExplainer(model, background)

# # for i in range(len(test_images)):
# for i in range(len(imgs)):
#     test = imgs[i].view(-1, 1, 28, 28)
#     shap_values = e.shap_values(test)

#     shap_numpy = [np.swapaxes(np.swapaxes(s, 1, -1), 1, 2) for s in shap_values]
#     test_numpy = np.swapaxes(np.swapaxes(test.numpy(), 1, -1), 1, 2)
    
#     out = model(test) # forward propagation
#     predicted = torch.max(out.data, 1)[1] # get predicted label from the maximum value
#     probs = F.softmax(out).detach() # get probabilities for each class
#     # pred_probs = probs[range(len(predicted)), predicted]  
#     pred_probs = probs[0][predicted] # get predicted probability for predicted label
    
#     print('--------\n')
#     print("Actual Class: {}, Predicted Class: {}, Predicted Probability: {:.4f}\n".format(
# #         labels_dict[test_labels[i].item()],
#         labels_dict[lbls[i].item()],
#         labels_dict[predicted.item()],
#         pred_probs.item()
        
#     ))
    
#     for cls in labels_dict.keys():
#         print('Prob for class {}: {:.4f}'.format(
#             labels_dict[cls],
#              probs[0][cls]))
    
#     shap.image_plot(shap_numpy, -test_numpy, labels = list(labels_dict.values()))

### Questions

**Q 1.1 Explain the output. What is displayed and what does it mean?**

...

---
---

# 2. Gradient-based methods

In gradient-based attribution methods, the gradients of the output (logits or soft-max probabilities) with respect to the extracted features or the input are calculated via backpropagation and are used to estimate attribution scores. Several variations of gradient-based approaches exist attempting to reduce the noise in the resulting attribution maps.

In [122]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

For loading the data and training the network, we loosely follow [this tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).

## Load CIFAR10 dataset

In [142]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

# load data
trainset = torchvision.datasets.CIFAR10(root='../datasets', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../datasets', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

# classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

classes = {
    0:'plane',
    1:'car',
    2:'bird',
    3:'cat',
    4:'deer',
    5:'dog',
    6:'frog',
    7:'horse',
    8:'ship',
    9:'truck'
}

### inspecting the data

In [ ]:
# function to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

## visualise some examples fromthe training data
# print labels
print([classes[labels[j].item()] for j in range(batch_size)])
# show images
imshow(torchvision.utils.make_grid(images))

## Training a neural network 

In [144]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### training the model

### &#9888;&#65039; &#9888;&#65039; If the training process takes too long on your machine, you can load the trained model from the repo:

In [145]:
random_seed = 1
torch.manual_seed(random_seed)

# define your model
model = Net()

# define the optimizer and loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# train model
for epoch in range(2):  # loop over the dataset multiple times
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        if i % 2000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch + 1, i * len(data), len(trainloader.dataset),
                100. * i / len(trainloader), loss.item()))

print('Finished Training')

In [16]:
# # save the trained model if desired
# PATH = '../03_unboxing/models/cifar_net_trained.pth'
# torch.save(model.state_dict(), PATH)

### evaluating on the test set

In [ ]:
# # load the trained model if necessary
# PATH = '../03_unboxing/models/cifar_net_trained.pth'
# model = Net()
# model.load_state_dict(torch.load(PATH))

In [ ]:
correct = 0
total = 0
model.eval()

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest value is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
    correct, len(testloader.dataset),
    100. * correct / len(testloader.dataset)))

### inspect prediction accuracy per class

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[label.item()] += 1
            total_pred[label.item()] += 1

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classes[classname]:5s} is {accuracy:.1f} %')

## Applying GradCAM

We are using the implementation from [this package](https://github.com/jacobgil/pytorch-grad-cam).

In [149]:
target_layer = model.conv2 # layer to visualize (last convolutional layer)

In [ ]:
from pytorch_grad_cam import GradCAM
import cv2

# initialize the GradCAM instance with your model
gradcam = GradCAM(model=model, target_layers=[target_layer])

# get some images
dataiter = iter(testloader)
images, labels = next(dataiter)
images, labels = next(dataiter)

# get predicted values
outputs = model(images)
_, predicted = torch.max(outputs, 1)
probs = F.softmax(outputs, dim=1)
# probs

# compute the GradCAM heatmaps
cam_list = []

for i, image in enumerate(images):
    image.requires_grad_()
    grayscale_cam = gradcam(input_tensor=image.unsqueeze(0))
    cam_list.append(grayscale_cam)
    
    image = image.cpu().permute(1, 2, 0).detach().numpy()
    heatmap = cam_list[i][0]  # assuming you only have one heatmap for each image
    
    # resize heatmap to match image dimensions
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    
    # convert heatmap to 8-bit unsigned integer (CV_8UC1) (required to apply colormap)
    heatmap = (heatmap * 255).astype('uint8')
    heatmap = cv2.applyColorMap(1-heatmap, cv2.COLORMAP_JET) # apply colormap
    # normalize heatmap
    vmin = heatmap.min()
    vmax = heatmap.max()
    heatmap = (heatmap - vmin)/(vmax - vmin)

    # overlay heatmap on image
    overlaid_img = heatmap * 0.5 + image * 0.5   # adjust the weight for overlay

    print('-------------------------------\n')
    print(f'Ground Truth: {classes[labels[i].item()]}')
    print(f'Predicted: {classes[predicted[i].item()]}\n')
    
    for cls in classes.keys():
        print('Prob for class {}: {:.4f}'.format(
            classes[cls],
             probs[i][cls]))
        
    im_tensor = torch.tensor(image.transpose((2, 0, 1)))
    heat_tensor = torch.tensor(heatmap.transpose((2, 0, 1)))
    overlay = torch.tensor(overlaid_img.transpose((2,0,1)))
    imshow(torchvision.utils.make_grid([im_tensor, heat_tensor, overlay]))

### Questions

**Q 2.1 Explain the output.**

...

**Q 2.2 Evaluate the explanations given by Gradcam.**

...